# Packages

In [2]:
#%cd /content/drive/MyDrive/MscThesis
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import os
import pandas as pd
import time
import gwpy
import nds2 # conda config --add channels conda-forge \n conda install python-nds2-client
import numba
import h5py
from gwpy.table import EventTable
from gwpy.table import GravitySpyTable
from gwpy.detector import ChannelList, Channel
from gwpy.time import tconvert
from gwpy.timeseries import TimeSeries #if this does not work, remove h5py and reinstall h5py
from matplotlib.colors import ListedColormap
from matplotlib.legend_handler import HandlerLine2D
from matplotlib.lines import Line2D
import matplotlib.cm as cm
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
from PIL import Image
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.optim as optim
import torchvision
import torchvision.models
import torchvision.transforms as transforms
from torch.optim import SGD, Adam, AdamW
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision.datasets import DatasetFolder, ImageFolder
from tqdm import tqdm
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, \
accuracy_score
from avalanche.models import SlimResNet18, MTSlimResNet18, SimpleCNN
from avalanche.models import as_multitask, IncrementalClassifier
from avalanche.models.utils import avalanche_model_adaptation
from avalanche.training import Naive, LwF, GenerativeReplay, ICaRLLossPlugin, ICaRL, EWC, AR1, LFL, AGEM
from avalanche.logging import InteractiveLogger, TextLogger, CSVLogger
from avalanche.training.plugins import EvaluationPlugin, ReplayPlugin, LwFPlugin, EarlyStoppingPlugin, AGEMPlugin
from avalanche.training.plugins.lr_scheduling import LRSchedulerPlugin
from avalanche.benchmarks import nc_benchmark, ni_benchmark
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.benchmarks.scenarios.dataset_scenario import benchmark_from_datasets
from avalanche.benchmarks.scenarios.supervised import class_incremental_benchmark
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics,loss_metrics, \
timing_metrics, cpu_usage_metrics, StreamConfusionMatrix, disk_usage_metrics, gpu_usage_metrics, \
confusion_matrix_metrics, bwt_metrics, forward_transfer_metrics, ram_usage_metrics, images_samples_metrics
from avalanche.evaluation.metrics import Accuracy, BWT, Forgetting, ForwardTransfer
import multiprocessing as mp

import my_utils
import my_architectures
import my_dataloaders
import my_gwpy_and_fractals
import my_transformations

IMG_SIZE = (224,224)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# calculating the amount of workers usable
number_of_workers = mp.cpu_count()
number_of_workers = int(number_of_workers/2)
print(number_of_workers)

4


In [3]:
# Set the environment variables
os.environ["GRAVITYSPY_DATABASE_USER"] = "mla"
os.environ["GRAVITYSPY_DATABASE_PASSWD"] = "gl1tch35Rb4d!"

In [4]:
current_dir = %pwd
print(current_dir)

C:\Users\Brian.Baert\Documents\GitHub\MscThesis


In [5]:
# Setting the correct directories
#train_dir = 'D:\\Users\\Brian\\TrainingSet_CL_1\\TrainingSet_CL_1'
train_dir = 'C:\\Users\\Brian.Baert\\TrainingSet_CL_1'
#train_dir_2 = 'D:\\Users\Brian\\TrainingSet_CL_2\\TrainingSet_CL_2'
train_dir_2 = 'C:\\Users\\Brian.Baert\\TrainingSet_CL_2'
#val_dir = 'D:\\Users\\Brian\\TestSet_CL\\TestSet_CL'
val_dir = 'C:\\Users\\Brian.Baert\\ValidationSet_CL'
#test_dir = 'D:\\Users\\Brian\\TestSet_CL\\TestSet_CL'
test_dir = 'C:\\Users\\Brian.Baert\\TestSet_CL'

# Read in class labels
class_file = open("classes.txt", "r")
classes = class_file.read()
classes = classes.split(", ")

In [6]:
O3b_csv = pd.read_csv('Data/data_o3b_high_confidence.csv')

In [7]:
FD_location = 'C:\\Users\\Brian.Baert\\FD_CL\\final_normalized_data.npy'

In [8]:
fractal_matrices = np.load(FD_location)
fractal_matrices.shape

(3584, 347, 56)

In [10]:
#pd_fractal = pd.DataFrame(fractal_matrices.reshape(fractal_matrices.shape[0], -1))
#pd_fractal

In [13]:
import numpy as np
from sklearn.decomposition import PCA

# Assuming X is your data array of shape (3584, 347, 56)
X_reshaped = fractal_matrices.reshape(fractal_matrices.shape[0], -1)  # Reshape to 2D

# Initialize PCA
# You can specify the number of components you want to keep
# For example, let's keep 10 components
pca = PCA(n_components=10)

# Fit and transform the data
X_pca = pca.fit_transform(X_reshaped)
X_pca.shape

# Now, X_pca is the transformed data, which is of shape (3584, 10)

(3584, 10)

In [218]:
no_glitch = fractal_matrices[0:895]
whistle = fractal_matrices[896:1791]
tomte = fractal_matrices[1792:2687]
scattered_light = fractal_matrices[2688:3584]

In [219]:
labels = []
labels.extend(['No_Glitch'] * 896)
labels.extend(['Whistle'] * 896)
labels.extend(['Tomte'] * 896)
labels.extend(['Scattered_Light'] * 896)
len(labels)
labels=np.array(labels)

In [220]:
f = h5py.File('C:\\Users\\Brian.Baert\\FD_CL\\FD_all_n=3584_2023-05-02.hdf5')
list(f)
for key in f['metadata'].keys():
    print(key)
 #3584 glitch calculations
for key in f['fd'].keys():
    print(key)
len(f['metadata']['channels'][:])

In [225]:
f['fd']['1238178793.877']

<HDF5 dataset "1238178793.877": shape (1913, 56), type "<f4">

In [315]:
# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(fractal_matrices, labels, test_size=0.2)

# Verify the shapes of the resulting arrays
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

X_train shape: (2867, 347, 56)
X_test shape: (717, 347, 56)
Y_train shape: (2867,)
Y_test shape: (717,)


In [ ]:
X_train.to

In [316]:
print(test_cluster_labels)
label_mapping = {0: 'No_Glitch', 1: 'Whistle', 2: 'Tomte', 3: 'Scattered_Light'}
# Replace values using the dictionary
test_labels = np.vectorize(label_mapping.get)(test_cluster_labels)
print(test_labels)

[3 0 0 ... 3 2 0]
['Scattered_Light' 'No_Glitch' 'No_Glitch' ... 'Scattered_Light'
 'Tomte' 'No_Glitch']


In [317]:
import torch
from sklearn.preprocessing import LabelEncoder
data = torch.from_numpy(X_train).float()  # Assuming data is a numpy array

# Create a label encoder
le = LabelEncoder()
# Fit the encoder on the training labels
le.fit(Y_train)
# Convert string labels to numerical labels
Y_train_encoded = le.transform(Y_train)
# Convert the encoded labels to a long tensor
labelsEnc = torch.from_numpy(Y_train_encoded).long()
labelsEnc.unique()

tensor([0, 1, 2, 3])

In [318]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

num_epochs=2

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data, label = self.data[idx], self.labels[idx]
        #data = data.unsqueeze(1)
        return data, label


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=347, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm1d(128)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32*56, 256)
        self.fc2 = nn.Linear(256,128)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(128,4)


    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Assuming your data and labels are loaded into 'data' and 'labels' variables
dataset = CustomDataset(data, labelsEnc)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)



def weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)
            
# Create an instance of the CNN architecture
model = Net()
model.apply(weights_init)

Net(
  (conv1): Conv1d(347, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv1d(64, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=1792, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (dropout2): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=128, out_features=4, bias=True)
)

In [319]:
x = torch.randn(16,347,56)
output = model(x)
print(output.shape)

torch.Size([16, 4])


In [324]:
import torch.optim as optim

# Define hyperparameters
learning_rate = 0.00001
epochs = 50
patience = 3
best_loss = float('inf')
best_epoch = 0
early_stop = False

criterion = nn.MSELoss()
# Define optimizer (e.g., Adam optimizer)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Loop through epochs
for epoch in range(epochs):
    # Loop through data batches
    for data_X, labels_Y in dataloader:
        # Clear gradients
        optimizer.zero_grad()
        print(f'Labels in this batch: {labels_Y.unique}')
        # Forward pass
        outputs = model(data_X)
        #print(outputs)
        predicted, _ = torch.max(outputs, 1)
        #print(predicted)
        loss = criterion(predicted, labels_Y.float())  # Convert labels to float for loss calculation

        # Backward pass and parameter update
        loss.backward()
        optimizer.step()

    scheduler.step()
    # Print training information (optional)
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

    # Check for early stopping
    if loss.item() < best_loss:
        best_loss = loss.item()
        best_epoch = epoch
    elif (epoch - best_epoch) >= patience:
        print(f'Early stopping at epoch {epoch+1}, best loss was {best_loss:.4f} at epoch {best_epoch+1}')
        early_stop = True
        break

if not early_stop:
    print('Finished training')

Labels in this batch: <bound method Tensor.unique of tensor([2, 1, 2, 3, 3, 2, 0, 3, 0, 1, 3, 1, 3, 0, 0, 2])>
Labels in this batch: <bound method Tensor.unique of tensor([1, 3, 3, 3, 0, 2, 0, 2, 3, 2, 3, 2, 3, 3, 3, 1])>
Labels in this batch: <bound method Tensor.unique of tensor([0, 2, 0, 2, 2, 1, 2, 2, 1, 1, 0, 3, 0, 3, 3, 3])>
Labels in this batch: <bound method Tensor.unique of tensor([0, 1, 2, 1, 1, 2, 3, 1, 1, 1, 3, 3, 2, 3, 1, 3])>
Labels in this batch: <bound method Tensor.unique of tensor([1, 0, 0, 1, 3, 2, 2, 3, 1, 1, 0, 1, 1, 3, 1, 1])>
Labels in this batch: <bound method Tensor.unique of tensor([3, 0, 1, 0, 1, 0, 2, 1, 3, 1, 1, 0, 2, 1, 0, 3])>
Labels in this batch: <bound method Tensor.unique of tensor([3, 3, 2, 0, 0, 3, 0, 3, 2, 2, 0, 2, 1, 2, 3, 0])>
Labels in this batch: <bound method Tensor.unique of tensor([2, 1, 2, 1, 0, 0, 3, 3, 1, 3, 2, 2, 3, 0, 3, 0])>
Labels in this batch: <bound method Tensor.unique of tensor([3, 0, 1, 1, 0, 3, 1, 3, 2, 1, 3, 3, 1, 1, 1, 2])>
L

In [321]:
y_pred_list, y_true_list = my_utils.get_predictions(model, dataloader)

get_predictions: 1240.3504848480225 ms


In [322]:
# GENERATE CONFUSION MATRIX AND F1
cm = confusion_matrix(y_true_list, y_pred_list)
f1 = f1_score(y_true_list, y_pred_list, average=None)

In [323]:
f1

array([0.        , 0.40411912, 0.        , 0.        ])